In [0]:
!pip install pgmpy wrapt

    100% |████████████████████████████████| 225kB 7.0MB/s 
  Running setup.py bdist_wheel for pgmpy ... - \ done
  Stored in directory: /root/.cache/pip/wheels/5e/5d/c5/81dd9fc173c4b56cc6f38b943d3d73b81f1096d67c52ae278a
  Running setup.py bdist_wheel for wrapt ... - \ done
  Stored in directory: /root/.cache/pip/wheels/48/5d/04/22361a593e70d23b1f7746d932802efe1f0e523376a74f321e
Successfully built pgmpy wrapt


# Creating Model

In [0]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianModel

accident_cpd = TabularCPD(variable='A',
                       variable_card=2,
                       values=[[.6, .4]])

game_cpd = TabularCPD(variable='G',
                       variable_card=2,
                       values=[[.5, .5]])

sunny_cpd = TabularCPD(variable='S',
                       variable_card=2,
                       values=[[.7, .3]])

rain_cpd = TabularCPD(variable='R',
                       variable_card=2,
                       values=[[.8, .2]])

new_dish_cpd = TabularCPD(variable='N',
                       variable_card=2,
                       values=[[.5, .5]])


traffic_cpd = TabularCPD(variable='T',
                         variable_card=2,
                         values=[[.8, .05, .1, .5 ],
                                [.2,.95, .9, .5]],
                         evidence=['A', 'G'],
                         evidence_card=[2, 2])

pedestrian_cpd = TabularCPD(variable='P',
                         variable_card=2,
                         values=[[.4, .9, .1, .8 ],
                                [.6,.1, .9, .2]],
                         evidence=['S', 'R'],
                         evidence_card=[2, 2])                
                                 
Restaurent_booking_cpd = TabularCPD(variable='RB',
                         variable_card=2,
                         values=[[.95, .6, .9, .5, .4, .05, .7, .3 ],
                                [.05, .4, .1, .5, .6, .95, .3, .7]],
                         evidence=['N', 'T','P'],
                         evidence_card=[2, 2, 2])           

In [0]:
# buildind model
mcg_model = BayesianModel([('A', 'T'),('G', 'T'), ('S', 'P'), ('R', 'P'),('T','RB'),('N','RB'),('P','RB')])

In [0]:
# adding cpds
mcg_model.add_cpds(accident_cpd, game_cpd, sunny_cpd, rain_cpd, new_dish_cpd, traffic_cpd, pedestrian_cpd, Restaurent_booking_cpd)



In [0]:
# Iterate over fraud_model.get_cpds()
for cpd in mcg_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)

CPD of A:
+-----+-----+
| A_0 | 0.6 |
+-----+-----+
| A_1 | 0.4 |
+-----+-----+
CPD of G:
+-----+-----+
| G_0 | 0.5 |
+-----+-----+
| G_1 | 0.5 |
+-----+-----+
CPD of S:
+-----+-----+
| S_0 | 0.7 |
+-----+-----+
| S_1 | 0.3 |
+-----+-----+
CPD of R:
+-----+-----+
| R_0 | 0.8 |
+-----+-----+
| R_1 | 0.2 |
+-----+-----+
CPD of N:
+-----+-----+
| N_0 | 0.5 |
+-----+-----+
| N_1 | 0.5 |
+-----+-----+
CPD of T:
+-----+-----+------+-----+-----+
| A   | A_0 | A_0  | A_1 | A_1 |
+-----+-----+------+-----+-----+
| G   | G_0 | G_1  | G_0 | G_1 |
+-----+-----+------+-----+-----+
| T_0 | 0.8 | 0.05 | 0.1 | 0.5 |
+-----+-----+------+-----+-----+
| T_1 | 0.2 | 0.95 | 0.9 | 0.5 |
+-----+-----+------+-----+-----+
CPD of P:
+-----+-----+-----+-----+-----+
| S   | S_0 | S_0 | S_1 | S_1 |
+-----+-----+-----+-----+-----+
| R   | R_0 | R_1 | R_0 | R_1 |
+-----+-----+-----+-----+-----+
| P_0 | 0.4 | 0.9 | 0.1 | 0.8 |
+-----+-----+-----+-----+-----+
| P_1 | 0.6 | 0.1 | 0.9 | 0.2 |
+-----+-----+-----+-----+--

In [0]:
from pgmpy.inference.base import Inference
from pgmpy.factors import factor_product

import itertools


class SimpleInference(Inference):
    def query(self, var, evidence):
        # self.factors is a dict of the form of {node: [factors_involving_node]}
        factors_list = set(itertools.chain(*self.factors.values()))
        product = factor_product(*factors_list)
        reduced_prod = product.reduce(evidence, inplace=False)
        reduced_prod.normalize()
        var_to_marg = set(self.model.nodes()) - set(var) - set([state[0] for state in evidence])
        marg_prod = reduced_prod.marginalize(var_to_marg, inplace=False)
        return marg_prod
    
infer = SimpleInference(mcg_model)
l1 = infer.query(var=['RB'], evidence=[('N', 1), ('T', 0), ('P', 1)])
print(l1);
l2 = infer.query(var=['RB'], evidence=[('N', 0), ('T', 1), ('P', 0)])
print(l2);
l3 = infer.query(var=['T'], evidence=[('A', 0), ('G', 0)])
print(l3);

+------+-----------+
| RB   |   phi(RB) |
|------+-----------|
| RB_0 |    0.0500 |
| RB_1 |    0.9500 |
+------+-----------+
+------+-----------+
| RB   |   phi(RB) |
|------+-----------|
| RB_0 |    0.9000 |
| RB_1 |    0.1000 |
+------+-----------+
+-----+----------+
| T   |   phi(T) |
|-----+----------|
| T_0 |   0.8000 |
| T_1 |   0.2000 |
+-----+----------+
